# Display GEE remote sensing data on ESRI webmaps
This notebook provides a basic workflow to allow for the display of remote sensing images available on Google Earth Engine as layers on ESRI webmaps. The webmap and layers holding these images should be created and saved before running this script. The basic process is:

1. Images are first curated and processed on GEE.
2. We then define visualization parameters and generate tiles and publicly accessible urls. 
3. These are then used to update the tile url attributes of existing layers on an existing ESRI webmap. 

This notebook demonstrates the curation and addition of NAIP data, but the process can be used for any image assets available on GEE, including NLCD, Sentinel-2, Sentinel-1, and SRTM Digital Elevation data.

## Setup

In [28]:
# import arc libraries
from arcgis.gis import GIS
from arcgis.mapping import WebMap

import pandas as pd

In [29]:
# import and authenticate gee library
import ee

# trigger an interactive session that will allow you to authenticate to an existing GEE account through a browser
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
# define some global variables
USER = '' # your ArcOnline account that can access organizational assets
PASSWORD = '' # password for above account
WEBMAP = 'NAIP_viewer' # title of the existing webmap to be updated

## GEE
Here we use Google Earth Engine to generate new tile urls for naip imagery

In [5]:
# get NAIP and TIGER catalogs
NAIP = ee.ImageCollection("USDA/NAIP/DOQQ")
TIGER = ee.FeatureCollection("TIGER/2018/States")


In [18]:
NAIP_dates = r"C:\Users\jczawlytko\code\LCCAA\NAIP_dates.csv"
df = pd.read_csv(NAIP_dates)
print(df.columns)

Index(['State', 'ST', 'STATEFP', 'T1', 'T2', 'T3'], dtype='object')


In [30]:
# define image visualization parameters
naip_viz_params = {
  'bands':['R', 'G', 'B'],
  'min': 0,
  'max': 200
}

In [67]:
# create a dictionary to hold tiles by state keyword
TILES = {}

# loop thru df

for index, row in df.iterrows():
    # get state name and T1 and T2 NAIP years
    State = row ['State']
    T1 = row['T1']
    T2 = row['T2']
    print(f"Generating tiles for {row ['State']}                ", end='\r')
    
    # get stat bounds
    st_bounds = TIGER.filterMetadata('NAME', 'equals', State)

    #create state naip collection then filter by T1 and T2 dates
    st_naip = NAIP.filterBounds(st_bounds)
    naipT1 = st_naip.filterDate(f'{T1}-01-01', f'{T1}-12-31').median()
    naipT2 = st_naip.filterDate(f'{T2}-01-01', f'{T2}-12-31').median()

    #create 
    map_id_dict1 = ee.Image(naipT1).getMapId(naip_viz_params)
    map_id_dict2 = ee.Image(naipT2).getMapId(naip_viz_params)
    tiles1 = map_id_dict1['tile_fetcher'].url_format
    tiles2 = map_id_dict2['tile_fetcher'].url_format
    TILES[State] = [tiles1, tiles2]
    
print(f"{len(TILES)} sets of tiles generated                        ")

14 sets of tiles generated                        


## ESRI webmap
In this section we update the tile urls attributes of the layers on an ESRI webmap. This webmap and layers should exist already  

In [33]:
# first we create a arconline client with our credentials

gis = GIS(username =  USER, password = PASSWORD)

In [34]:
# find the instance of the webmap we want to display gee imagery on
search_results = gis.content.search(query = f'title:{WEBMAP}', item_type = 'Web Map')
wm_item = search_results[0]

In [ ]:
# cast the existing webmap into a WebMap class object
web_map_obj = WebMap(wm_item)

In [62]:
# obtain the layers that will display NAIP data
layers = web_map_obj.layers
naip_layers = [layer for layer in layers if 'NAIP' in layer['title']]
naip_layers

In [ ]:
for state, tiles in TILES.items():
    layers = [naip_layer for naip_layer in naip_layers if state in naip_layer['title']]
    [layers[i].update({'templateUrl':tile}) for i, tile in enumerate( tiles)]

In [40]:
web_map_obj.update()

True